首先，在需要使用到回调的代码中，关键点在于当前计算工作会挂起并在将来的某个时候重启(比如异步执行)。
当计算重启时，回调函数被调用来继续处理结果。apply_async() 函数演示了执行回调的实际逻辑，
尽管实际情况中它可能会更加复杂(包括线程、进程、事件处理器等等)。

计算的暂停与重启思路跟生成器函数的执行模型不谋而合。
具体来讲，yield 操作会使一个生成器函数产生一个值并暂停。
接下来调用生成器的 __next__() 或 send() 方法又会让它从暂停处继续执行。

根据这个思路，这一小节的核心就在 inline_async() 装饰器函数中了。
关键点就是，装饰器会逐步遍历生成器函数的所有 yield 语句，每一次一个。
为了这样做，刚开始的时候创建了一个 result 队列并向里面放入一个 None 值。
然后开始一个循环操作，从队列中取出结果值并发送给生成器，它会持续到下一个 yield 语句，
在这里一个 Async 的实例被接受到。然后循环开始检查函数和参数，并开始进行异步计算 apply_async() 。
然而，这个计算有个最诡异部分是它并没有使用一个普通的回调函数，而是用队列的 put() 方法来回调。

这时候，是时候详细解释下到底发生了什么了。主循环立即返回顶部并在队列上执行 get() 操作。
如果数据存在，它一定是 put() 回调存放的结果。如果没有数据，那么先暂停操作并等待结果的到来。
这个具体怎样实现是由 apply_async() 函数来决定的。

In [1]:
from queue import Queue
from functools import wraps

def apply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)

    # Invoke the callback with the result
    callback(result)
    
class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args

def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None)
        while True:
            result = result_queue.get()
            try:
                a = f.send(result)
                apply_async(a.func, a.args, callback=result_queue.put)
            except StopIteration:
                break
    return wrapper

def add(x, y):
    return x + y

@inlined_async
def test():
    r = yield Async(add, (2, 3))
    print(r)
    r = yield Async(add, ('hello', 'world'))
    print(r)
    for n in range(10):
        r = yield Async(add, (n, n))
        print(r)
    print('Goodbye')

In [6]:
test()

5
helloworld
0
2
4
6
8
10
12
14
16
18
Goodbye


代码展示了一个使用生成器和回调函数来实现异步编程的例子。让我们一步一步解释这个例子的执行过程：

- apply_async函数：
这是一个模拟异步操作的函数。它接受一个函数 func 和参数 args，并通过调用 func(*args) 来计算结果。然后，它调用传递的回调函数 callback 并将计算的结果传递给它。

- Async类：
这个类用于封装函数和参数，方便后续传递给异步函数。

- inlined_async装饰器：
这是一个装饰器函数，用于将一个使用生成器的函数转换成异步函数。在装饰的函数内部，它会创建一个生成器 f，然后通过调用 f.send(result) 来向生成器发送结果。生成器会接收到结果，并根据结果继续执行。

- add函数：
这是一个简单的加法函数。

- test函数（被装饰的函数）：
在这个函数中，我们使用了 inlined_async 装饰器来将它转换成异步函数。在函数内部，我们使用了 yield 关键字来暂停函数的执行，并等待异步操作的结果。

第一个异步操作：r = yield Async(add, (2, 3))，我们创建了一个 Async 对象来表示异步操作，然后通过 yield 暂停函数的执行，并等待异步操作的结果。一旦异步操作完成，结果会通过生成器的 send 方法返回，并赋值给 r。

后续异步操作：通过类似的方式，我们执行了其他异步操作。

异步操作结果的处理：通过 print(r) 来打印异步操作的结果。

StopIteration 异常：当所有异步操作都完成时，生成器会触发 StopIteration 异常，从而跳出循环，函数执行结束。

- 最后，我们调用 test() 函数来运行异步操作。

需要注意的是，这个例子中的异步操作并不是真正的并发执行，而是通过生成器的 yield 和回调函数模拟出异步的效果。在实际应用中，如果需要实现真正的并发，可能需要使用异步库（如asyncio）或多线程/多进程来实现。